In [7]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

file_path = "../Input_Files/UA_UB_Rates_2025-01-01.xlsx"
df = pd.read_excel(file_path, skiprows=1)
# Drop the last row
df = df.iloc[:-1]
file_name = file_path.split('/')[-1]  # Extract the file name from the path

# Extract numbers from the file name using list comprehension
numbers_str = ''.join([char for char in file_name if char.isdigit()])

# Convert numbers_str to datetime object
date_obj = datetime.strptime(numbers_str, '%Y%m%d')

# Format date_obj to 'YYYY-MM-DD'
final_date = date_obj.strftime('%Y-%m-%d')

# Create a directory named after the date if it doesn't exist
if not os.path.exists(f'../Output_Files/final/{final_date}'):
    os.makedirs(f'../Output_Files/final/{final_date}')    

#Import Rejected Codes
df_rej_med = pd.read_excel('../Input_Files/MCal_UAUB_Medical_Rejected.xlsx', header=None, names=['Proc_Code'], skiprows=1)
df_rej_op = pd.read_excel('../Input_Files/MCal_UAUB_Outpatient_Rejected.xlsx', header=None, names=['Proc_Code'], skiprows=1)
df_rej = pd.read_excel('../Input_Files/MCal_UAUB_Rejected.xlsx', header=None, names=['Proc_Code'], skiprows=1)

# Define new column names based on the data structure in the file
df.columns = ['OP_Proc_From', 'OP_Proc_Through', 'Outpatient_UA', 'Outpatient_UB',
              'Med_Proc_From', 'Med_Proc_Through', 'Medical_UA', 'Medical_UB']
#display(df)
df['ASOFDATE'] = numbers_str


# Change data types to 'str' for the from and through columns
df['OP_Proc_From'] = df['OP_Proc_From'].astype('str')
df['OP_Proc_Through'] = df['OP_Proc_Through'].astype('str')
df['Med_Proc_From'] = df['Med_Proc_From'].astype('str')
df['Med_Proc_Through'] = df['Med_Proc_Through'].astype('str')

# Split the DataFrame into df_OP and df_med
df_OP = df[['OP_Proc_From', 'OP_Proc_Through', 'Outpatient_UA', 'Outpatient_UB','ASOFDATE']]
df_med = df[['Med_Proc_From', 'Med_Proc_Through', 'Medical_UA', 'Medical_UB','ASOFDATE']]

#For Out Patients
# Use regular expressions to split alphanumeric characters
df_OP['OP_Proc_From_Num'] = df_OP['OP_Proc_From'].str.extract('(\d+)', expand=False)
df_OP['OP_Proc_From_Pref'] = df_OP['OP_Proc_From'].apply(lambda x: x[0] if x[0].isalpha() else np.nan)
df_OP['OP_Proc_From_Suff'] = df_OP['OP_Proc_From'].apply(lambda x: x[-1] if x[-1].isalpha() else np.nan)

df_OP['OP_Proc_Through_Num'] = df_OP['OP_Proc_Through'].str.extract('(\d+)', expand=False)
df_OP['OP_Proc_Through_Pref'] = df_OP['OP_Proc_Through'].apply(lambda x: x[0] if x[0].isalpha() else np.nan)
df_OP['OP_Proc_Through_Suff'] = df_OP['OP_Proc_Through'].apply(lambda x: x[-1] if x[-1].isalpha() else np.nan)

# Create a new DataFrame to store the expanded rows
df_expanded = pd.DataFrame()

# Determine the length of the 'From' and 'Through' numbers
num_length = len(df_OP['OP_Proc_From_Num'].iloc[0])

# Iterate over each row in the original DataFrame
for _, row in df_OP.iterrows():
    # Convert the 'From' and 'Through' numbers to integers
    from_num = int(row['OP_Proc_From_Num'])
    through_num = int(row['OP_Proc_Through_Num'])

    # Create a new DataFrame for this row
    df_row = pd.DataFrame()

    # Iterate over the range from 'From' to 'Through'
    for num in range(from_num, through_num + 1):
        # Create a copy of the row
        new_row = row.copy()

        # Update the 'From' and 'Through' numbers
        new_row['OP_Proc_From_Num'] = str(num).zfill(num_length)
        
        # Append the new row to the DataFrame for this row
        df_row = pd.concat([df_row, pd.DataFrame(new_row).T])

    # Append the DataFrame for this row to the expanded DataFrame
    df_expanded = pd.concat([df_expanded, df_row])

# Reset the index of the expanded DataFrame
df_expanded.reset_index(drop=True, inplace=True)

# Create a new column 'OP_Proc_From_new' by concatenating 'OP_Proc_From_Pref', 'OP_Proc_From_Num', and 'OP_Proc_From_Suff'
df_expanded['OP_Proc_From_new'] = df_expanded['OP_Proc_From_Pref'].fillna('') + df_expanded['OP_Proc_From_Num'].fillna('') + df_expanded['OP_Proc_From_Suff'].fillna('')


df_OP_UAUB = df_expanded[['OP_Proc_From_new', 'OP_Proc_Through', 'Outpatient_UA', 'Outpatient_UB','ASOFDATE']]
df_OP_UAUB.rename(columns={'OP_Proc_From_new': 'OP_Proc_From'}, inplace=True)

#merged_OP_df = pd.merge(df_OP_UAUB, df_rej_op, how='outer', left_on='OP_Proc_From', right_on='Proc_Code', indicator=True)
#OP_df = merged_OP_df[merged_OP_df['_merge'] == 'left_only']
#OP_df.pop('_merge')
#OP_df.pop('Proc_Code')


OP_df = df_OP_UAUB[~df_OP_UAUB['OP_Proc_From'].isin(df_rej['Proc_Code'])]


# Create a dataframe for modifier 'UA'
df_ua_op = pd.DataFrame({
    'SVC_CD': OP_df['OP_Proc_From'],
    'MODIFIER_1': 'UA',
    'FEE': OP_df['Outpatient_UA'],
    'ASOFDATE': OP_df['ASOFDATE']
})

# Create a dataframe for modifier 'UB'
df_ub_op = pd.DataFrame({
    'SVC_CD': OP_df['OP_Proc_From'],
    'MODIFIER_1': 'UB',
    'FEE': OP_df['Outpatient_UB'],
    'ASOFDATE': OP_df['ASOFDATE']
})

# Concatenate the two dataframes
result_df_op = pd.concat([df_ua_op, df_ub_op], ignore_index=True)
result_df_op.sort_values(by='SVC_CD', inplace=True, ignore_index=True)
display(result_df_op)



#For Med_Codes

# Use regular expressions to split alphanumeric characters
df_med['Med_Proc_From_Num'] = df_med['Med_Proc_From'].str.extract('(\d+)', expand=False)
df_med['Med_Proc_From_Pref'] = df_med['Med_Proc_From'].apply(lambda x: x[0] if x[0].isalpha() else np.nan)
df_med['Med_Proc_From_Suff'] = df_med['Med_Proc_From'].apply(lambda x: x[-1] if x[-1].isalpha() else np.nan)

df_med['Med_Proc_Through_Num'] = df_med['Med_Proc_Through'].str.extract('(\d+)', expand=False)
df_med['Med_Proc_Through_Pref'] = df_med['Med_Proc_Through'].apply(lambda x: x[0] if x[0].isalpha() else np.nan)
df_med['Med_Proc_Through_Suff'] = df_med['Med_Proc_Through'].apply(lambda x: x[-1] if x[-1].isalpha() else np.nan)


# Create a new DataFrame to store the expanded rows
df_med_expanded = pd.DataFrame()

# Determine the length of the 'From' and 'Through' numbers
num_length = len(df_med['Med_Proc_From_Num'].iloc[0])

# Iterate over each row in the original DataFrame
for _, row in df_med.iterrows():
    # Convert the 'From' and 'Through' numbers to integers
    from_num = int(row['Med_Proc_From_Num'])
    through_num = int(row['Med_Proc_Through_Num'])

    # Create a new DataFrame for this row
    df_row = pd.DataFrame()

    # Iterate over the range from 'From' to 'Through'
    for num in range(from_num, through_num + 1):
        # Create a copy of the row
        new_row = row.copy()

        # Update the 'From' and 'Through' numbers
        new_row['Med_Proc_From_Num'] = str(num).zfill(num_length)
        
        # Append the new row to the DataFrame for this row
        df_row = pd.concat([df_row, pd.DataFrame(new_row).T])

    # Append the DataFrame for this row to the expanded DataFrame
    df_med_expanded = pd.concat([df_med_expanded, df_row])

# Reset the index of the expanded DataFrame
df_med_expanded.reset_index(drop=True, inplace=True)

# Create a new column 'Med_Proc_From_new' by concatenating 'Med_Proc_From_Pref', 'Med_Proc_From_Num', and 'Med_Proc_From_Suff'
df_med_expanded['Med_Proc_From_new'] = df_med_expanded['Med_Proc_From_Pref'].fillna('') + df_med_expanded['Med_Proc_From_Num'].fillna('') + df_med_expanded['Med_Proc_From_Suff'].fillna('')

df_Med_UAUB = df_med_expanded[['Med_Proc_From_new', 'Med_Proc_Through', 'Medical_UA', 'Medical_UB','ASOFDATE']]
df_Med_UAUB.rename(columns={'Med_Proc_From_new': 'Med_Proc_From'}, inplace=True)

#merged_med_df = pd.merge(df_Med_UAUB, df_rej_med, how='outer', left_on='Med_Proc_From', right_on='Proc_Code', indicator=True)
#med_df = merged_med_df[merged_med_df['_merge'] == 'left_only']
#med_df.pop('_merge')
#med_df.pop('Proc_Code')
med_df = df_Med_UAUB[~df_Med_UAUB['Med_Proc_From'].isin(df_rej['Proc_Code'])]
# Create a dataframe for modifier 'UA'
df_ua_med = pd.DataFrame({
    'SVC_CD': med_df['Med_Proc_From'],
    'MODIFIER_1': 'UA',
    'FEE': med_df['Medical_UA'],
    'ASOFDATE': med_df['ASOFDATE']
})

# Create a dataframe for modifier 'UB'
df_ub_med = pd.DataFrame({
    'SVC_CD': med_df['Med_Proc_From'],
    'MODIFIER_1': 'UB',
    'FEE': med_df['Medical_UB'],
    'ASOFDATE': med_df['ASOFDATE']
})

# Concatenate the two dataframes
result_df_med = pd.concat([df_ua_med, df_ub_med], ignore_index=True)
result_df_med.sort_values(by='SVC_CD', inplace=True, ignore_index=True)
display(result_df_med)

#Write the files to csv
result_df_op.to_csv(f'../Output_Files/final/{final_date}/MCal_UAUB_Outpatient.{final_date}.txt', sep='|', index=False)
result_df_med.to_csv(f'../Output_Files/final/{final_date}/MCal_UAUB_Medical.{final_date}.txt', sep='|', index=False)
print('files saved successfully')

C:\Users\skrishnan\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\skrishnan\AppData\Local\Temp\ipykernel_13292\2132378195.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_OP['OP_Proc_From_Num'] = df_OP['OP_Proc_From'].str.extract('(\d+)', expand=False)
C:\Users\skrishnan\AppData\Local\Temp\ipykernel_13292\2132378195.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

,SVC_CD,MODIFIER_1,FEE,ASOFDATE
0,0499T,UA,30.0,20250101
1,0499T,UB,60.0,20250101
2,0581T,UB,35.0,20250101
3,0581T,UA,10.0,20250101
4,0660T,UB,30.0,20250101
...,...,...,...,...
11999,S2118,UB,130.0,20250101
12000,S2230,UB,70.0,20250101
12001,S2230,UA,40.0,20250101
12002,S2235,UA,40.0,20250101


C:\Users\skrishnan\AppData\Local\Temp\ipykernel_13292\2132378195.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_med['Med_Proc_From_Num'] = df_med['Med_Proc_From'].str.extract('(\d+)', expand=False)
C:\Users\skrishnan\AppData\Local\Temp\ipykernel_13292\2132378195.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_med['Med_Proc_From_Pref'] = df_med['Med_Proc_From'].apply(lambda x: x[0] if x[0].isalpha() else np.nan)
C:\Users\skrishnan\AppData\Local\Temp\ipykernel_13292\2132378195.py:133: Setti

,SVC_CD,MODIFIER_1,FEE,ASOFDATE
0,0499T,UA,35.0,20250101
1,0499T,UB,65.0,20250101
2,0581T,UB,40.0,20250101
3,0581T,UA,15.0,20250101
4,0660T,UB,35.0,20250101
...,...,...,...,...
11999,S2118,UB,135.0,20250101
12000,S2230,UB,75.0,20250101
12001,S2230,UA,45.0,20250101
12002,S2235,UA,45.0,20250101


files saved successfully
